# 4. YOLO RarePlanes Model Scoring

This last notebook converts the ground truth annotations to the bounding box format whcih YOLO creates in order to compare them with the predictions. Then, precision, recall, and f1 are calculated by class and as a whole. 

In [1]:
from shapely.geometry import box
import argparse
import geopandas as gpd
import pandas as pd
import os
import glob
from tqdm.notebook import tqdm
import numpy as np
from solaris.eval.iou import calculate_iou
from solaris.eval.vector import mF1

/home/ubuntu/anaconda3/envs/solaris/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/ubuntu/anaconda3/envs/solaris/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/ubuntu/anaconda3/envs/solaris/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/ubuntu/anaconda3/envs/solaris/lib/pyt

### Convert Ground Truth to Bounding Boxes

The following scripts take the geojson ground truths for the test set and convert them to bounding boxes. Then, in the two cells, the same custom classes are created in the ground truth. 

In [6]:
input_folder = "/home/ubuntu/src/yolo_planes/wdata/test/geojson_aircraft"
output_folder = "/home/ubuntu/src/yolo_planes/wdata/test/geojson_aircraft_bbox"
os.makedirs(output_folder,exist_ok=True)
os.chdir(input_folder)
geojsons = glob.glob("*.geojson")
for geojson in tqdm(geojsons):
    gdf = gpd.read_file(geojson)
    box_geoms= []
    for _, row in gdf.iterrows():
        x = row.geometry.bounds
        bbox = box(x[0], x[1], x[2], x[3])
        box_geoms.append(bbox)
    gdf['box_geom'] = box_geoms
    gdf = gdf.drop('geometry',axis=1)
    gdf = gpd.GeoDataFrame(gdf, geometry='box_geom')
    out_name=os.path.join(output_folder, geojson)
    gdf.to_file(out_name, driver='GeoJSON', encoding='utf-8')

100%|██████████| 66/66 [00:03<00:00, 16.67it/s]


In [14]:
def count_unique_index(df, by):
    return df.groupby(by).size().reset_index().rename(columns={0: 'count'})

def create_custom_classes(all_annotations_geojson, geojson_dir, output_path, category_attributes):
    """ parse the geojson files and create custom classes based upon
    unique variatons of the RarePlanes attributes.
        -all_annotations_geojson (str): The path to the
        `RarePlanes_Public_All_Annotations.geojson` file.
        - geojson_dir (str): directory containing the geojson files
        for individual images or tiles
        - output_path (str): directory to output the customized geojsons. Need to provide the absolute path.
        - category_attributes (list): A list of attributes to combine
        to create a custom class.  Choose any combintaion of the following:
        ['role','num_engines', 'propulsion', 'canards', 'num_tail_fins',
       'wing_position', 'wing_type', 'faa_wingspan_class']
    :returns
        - new geojsons with a custom_id for each combination of unique
        attributes.
        -A lookup table for each classes custom_id.
    """
    os.makedirs(output_path, exist_ok=True)
    gdf = gpd.read_file(all_annotations_geojson)
    lookup_gdf = count_unique_index(gdf, category_attributes)
    lookup_gdf['custom_id'] = list(range(0, len(lookup_gdf)))
    lookup_gdf.drop(columns=['count'], inplace=True)
    lookup_gdf.to_csv(os.path.join(output_path, "custom_class_lookup.csv"))
    os.chdir(geojson_dir)
    geojsons = glob.glob("*.geojson")
    for geojson in tqdm(geojsons):
        gdf = gpd.read_file(geojson)
        gdf = pd.merge(gdf, lookup_gdf, on=category_attributes, how='left')
        gdf["custom_id"] = pd.to_numeric(gdf["custom_id"], downcast='float')
        gdf.to_file(os.path.join(output_path, geojson), driver="GeoJSON", encoding='utf-8')

In [15]:
all_annotations_geojson = '/home/ubuntu/src/yolo_planes/wdata/RarePlanes_Public_All_Annotations.geojson'
geojson_dir_test = '/home/ubuntu/src/yolo_planes/wdata/test/geojson_aircraft_bbox'

output_path_test_one = '/home/ubuntu/src/yolo_planes/wdata/test/yolo_class_one_truth_bbox'

class_one = ['num_engines', 'propulsion']

create_custom_classes(all_annotations_geojson, geojson_dir_test, output_path_test_one, class_one)

100%|██████████| 66/66 [00:03<00:00, 18.73it/s]


### Results

The following cells execute the scoring of the predicitons vs the ground truth. The first cell outputs total recall, precision, and F1, while the second cell outputs the lookup table and now includes the class by class recall, precision, and F1. 

In [45]:
proposal_polygons_dir ="/home/ubuntu/src/yolo_planes/yolov5/inference/class_one_out/bounding_boxes"
gt_polygons_dir = "/home/ubuntu/src/yolo_planes/wdata/test/yolo_class_one_truth_bbox"
mF1, f1s_by_class, precision_iou_by_obj, precision_by_class, mPrecision, recall_iou_by_obj, recall_by_class, mRecall, object_subset, prop_objs, all_objs = mF1(proposal_polygons_dir, gt_polygons_dir, prediction_cat_attrib="class_id", gt_cat_attrib='custom_id', object_subset=[], all_outputs=True)

getting unique objects...
calculating recall...
mRecall: 0.6511768568429169
calculating precision...
mPrecision: 0.7047816872010533

calculating F1 scores...
mF1: 0.6732098637317354


/home/ubuntu/anaconda3/envs/solaris/lib/python3.7/site-packages/ipykernel_launcher.py:26: RuntimeWarning: Mean of empty slice
/home/ubuntu/anaconda3/envs/solaris/lib/python3.7/site-packages/ipykernel_launcher.py:364: RuntimeWarning: invalid value encountered in double_scalars


In [46]:
lookup_table = "/home/ubuntu/src/yolo_planes/geojsons_test/yolo_class_one/custom_class_lookup.csv"
lookup_table = pd.read_csv(lookup_table)
lookup_table.drop(lookup_table.columns[lookup_table.columns.str.contains('unnamed',case = False)],axis = 1, inplace = True)
object_subset_int = [int(i) + 1 for i in object_subset]
out_dict = {"custom_id":object_subset_int,"f1":f1s_by_class,"precision": precision_by_class, "recall": recall_by_class}
out_df = pd.DataFrame.from_dict(out_dict)
out_df = pd.merge(out_df, lookup_table, on='custom_id', how='left')
out_df

   num_engines propulsion  recall_by_class  precision_by_class  f1s_by_class
0            0  unpowered         0.729730            0.729730      0.729730
1            1        jet         0.250000            0.250000      0.250000
2            1  propeller         0.933941            0.933941      0.933941
3            2        jet         0.978015            0.978015      0.978015
4            2  propeller         0.885305            0.885305      0.885305
5            3        jet         0.000000            0.000000      0.000000
6            4        jet         0.690000            0.690000      0.690000
7            4  propeller         0.742424            0.742424      0.742424


Now, using these fused geojsons, you can visulaize the predictions by downloading the `/home/ubuntu/src/yolo_planes/yolov5/inference/class_one_out/bounding_boxes` folder to your local machine and using a geographic infromation system like QGIS and overlaying the bounding boxes onto the real image.